In [229]:
from IPython.display import display, display_html, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from threading import Thread
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Group Assignment
### Team Number: XX
### Team Member Names: Rehan, Rui, Anton
### Team Strategy Chosen: RISKY

*Delete before submission*

IMPORTANT SPECIFICATIONS DURING CODING:

- Stocks must be in US Market

- Stocks must have an average monthly volume of at least 200 000 shares between January 01, 2022 to October 31, 2022.

- Months must have at least 20 trading days

- Must have a minimum of 12 stocks and a maximum of 25 stocks

- If we choose n stocks for our portfolio, each stock must be minimum of (100/(2n))% of the portfolio when weighted by value (i.e., the overall value of the shares purchased in that particular stock) as of closing prices on November 25, 2022

- No individual stock may make up more than 25% of the portfolio when weighted by value (i.e., the overall value of the shares purchased in that particular stock) as of closing prices on November 26, 2022.

- Must spend all 500 000 USD on portfolio

- Teams will purchase their stocks at the closing prices on November 25, 2022

There are more specifications for how our code should be presented before submission, look into the assignment doc for info.

### Global Variables

In [230]:
# Total Investment Size
global_investment_size = 500000

# File Name
file = 'Tickers_Example.csv'

# Start date and end date for our portfolio
start_date = '2022-01-01'
end_date = '2022-11-01'


### Extracting tickers and adding them to a list

In [231]:
# Initializing a dataframe for 'raw' data extracted from the .csv file
tickers_raw = pd.read_csv(file, header=None)[0].tolist()

# Empty data structures to store ticker data in
tickers = []
tickers_hist = {}

# Function which consumes a ticker and determines the validation based on prerequisites
def validate_ticker(ticker):

    # Extracting ticker info from yFinance
    ticker_info = yf.Ticker(ticker).info

    # Trying every stock and excepting those that throw an error
    try:
        # If the stock is valid, we check for each prerequisite:
        # Checking for USD currency and ensuring it's on the US market
        if ticker_info['currency'] == 'USD' and ticker_info['market'] == 'us_market':
            ticker_hist = yf.Ticker(ticker).history(start=start_date, end=end_date, interval='1d').dropna()

            # Checking monthly volume
            ticker_monthly_trading_days = ticker_hist['Volume'].groupby(pd.Grouper(freq='MS')).count()
            ticker_monthly_volume = ticker_hist['Volume'].groupby(pd.Grouper(freq='MS')).sum()

            # Checking if the month has at least 20 trading days
            for month in ticker_monthly_trading_days.index:
                if ticker_monthly_trading_days.loc[month] < 20:
                    ticker_monthly_volume.drop(month, inplace=True)

            # Checking if the average monthly volume is greater than or equal to 200,000 USD
            if ticker_monthly_volume.mean() >= 200000:
                tickers.append(ticker)
                tickers_hist[ticker] = ticker_hist
            else:
                print(f'{ticker} Ticker does not meet average monthly volume requirements')
        else:
            print(f'{ticker} Ticker does not reference stock denominated in USD')
    except:
        print(f'Error: {ticker} Ticker does not reference a valid stock')

# Empty data structure for threading
threads = []

# Checking validity of each ticker in list of tickers given from threading
for ticker in tickers_raw:
    thread = Thread(target=validate_ticker, args=[ticker])
    thread.start()
    threads.append(thread)

# Using threading
for thread in threads:
    thread.join()

### List of Valid Tickers

In [ ]:
print(tickers)

['C', 'COST', 'ACN', 'AIG', 'BAC', 'BA', 'MON', 'LMT', 'MS', 'AXP', 'CAT', 'TGT', 'MO', 'BK', 'ABBV', 'SLB', 'SO', 'TXN', 'BLK', 'BMY', 'USB', 'CVS', 'PFE', 'SPG', 'NEE', 'CMCSA', 'GOOG', 'UPS', 'LLY', 'KMI', 'AMZN', 'UNH', 'PG', 'AAPL', 'COP', 'T', 'PM', 'SBUX', 'JPM', 'ORCL', 'COF', 'MRK', 'CSCO', 'CL', 'BIIB', 'QCOM', 'ABT', 'GM', 'MSFT', 'PYPL', 'OXY', 'NKE', 'UNP', 'KO', 'PEP']


---

## Strategy and Data Analytics

The strategy our team has chosen is to go risky, meaning we must optimize a portfolio so that it is driven away as much as possible, from the initial starting value of $500,000. Since we are pursuing a risky strategy, there are a few factors to an optimal portfolio.

Since we want the most risk and most reward, we want to minimize any diversification involved. Essentially, out of the minimum and maximum requirements for stock diversification, we want to pick the least amount of stocks, which is a minimum of 12 and keep diversification at an all time low. Furthermore, our stocks can net heavy volume, however, volume is useless if the market direction for these assets is all over the place. Therefore, we need all of our stocks to move in the same direction. 

Overall, we will move forward with the minimum amount of stocks involved, 12, and keep track of various factors like risk to reward ratio, positive risk, negative risk, betas and options markets.

### Measuring Beta

Beta calculates the volatility of a stock given the covariance of the stock relative to the broader stock market and the variance of the stock. We can use this to our advantage and find the higher volatility stocks so that we can create a riskier portfolio. The formula for Beta is as follows:

$$
\beta_i = \frac {\mathrm{Cov} (r_i,r_m)}{\mathrm{Var} (r_m)}
$$


Where $ \beta_i $ is the market beta of a stock, $ {\mathrm{Cov} (r_i,r_m)} $ is the covariance between the stock and the market index (in our case, the S&P 500), $ {\mathrm{Var} (r_m)} $ is the variance of the market index. Since a volatile stock can net us a gain or a loss, we want the highest magnitude. Thus, we will calculate both an upper bound beta and a lower bound beta and yield the highest magnitude out of the two as a contender for our final portfolio

In [ ]:
# We will be using S&P 500 as our measure of how the overall stock market is performing
sp_index = yf.Ticker("^GSPC") # ticker for S&P 500

# Extracting close prices of the S&P 500, percentage returns and its corresponding variance
sp_hist = sp_index.history(start=start_date, end=end_date).filter(like="Close")
sp_hist["Returns"] = sp_hist['Close'].pct_change()
sp_var = sp_hist["Returns"].var()

In [ ]:
frame = {}
beta_values = []
min_stocks = 12
max_stocks = 25

market_index = yf.Ticker('^GSPC').history(start=start_date, end=end_date, interval='1d').dropna()['Close']
market_return = market_index.pct_change()
market_variance = market_return.var()

beta_dict = {}

for ticker in tickers:
    stock_return = tickers_hist[ticker]['Close'].pct_change()
    covariance = pd.DataFrame({ticker: stock_return, 'S&P 500': market_return}).cov().loc[ticker, 'S&P 500']
    beta_dict[ticker] = covariance / market_variance

beta_df = pd.DataFrame(beta_dict.items(), columns=['Ticker', 'Beta'])
beta_df.sort_values(by=['Beta'], ascending=False, ignore_index=True, inplace=True)

### The portfolio with the highest magnitude of beta is printed below ###

In [ ]:
# Long Term Investment (TEMPORARY BACKUP)
highest_betas_magn



In [ ]:
# Finding the higher magnitude portfolio as our beta portfolio
if (abs(highest_betas_magn) > abs(lowest_betas_magn)):
    beta_portfolio = highest_betas
else:
    beta_portfolio = lowest_betas

beta_portfolio_print = beta_portfolio
beta_portfolio = beta_portfolio.reset_index() # resetting index to use in a for loop

# Putting the tickers into a list to calculate investment amounts later
beta_tickers = []

for i in range(len(beta_portfolio.index)):
    beta_tickers.append(beta_portfolio['Ticker'][i])

beta_portfolio_print

,Ticker,Beta
31,PYPL,1.651318
20,AMZN,1.604552
39,QCOM,1.454487
14,GM,1.393548
11,GOOG,1.310727
12,NKE,1.300717
46,BA,1.298421
17,BLK,1.269739
51,AAPL,1.265965
21,MSFT,1.247844


---

### Optimal Weighting for Stocks

Since we want the highest volatility possible, we will invest the majority of our funds into the most volatile stock(s) before investing into the others. Given the requirements of a stock only being a maximum of 25% of the entire portfolio, we have calculated the optimal weighting below to create the most volatility.

In [ ]:
min_weighting = 100 / (min_stocks * 2)
max_weighting = 25
list_of_weighting = []
total_weighting = 0 #total_weighting will need to add up to 100 in the end
num_stocks = min_stocks


# We want to optimize the weighting of each stock such that the more volatile stocks take up the majority of the portfolio
while (num_stocks != 0):
    temp_weighting = min_weighting * (num_stocks - 1)
    remaining_weight = 100 - (temp_weighting + total_weighting)
    
    if (remaining_weight > max_weighting):
        list_of_weighting.append(max_weighting/100)
        total_weighting += max_weighting
    elif (remaining_weight > min_weighting):
        list_of_weighting.append(remaining_weight/100)
        total_weighting += remaining_weight
    else:
        list_of_weighting.append(min_weighting/100)
        total_weighting += min_weighting
    num_stocks -= 1

---

## Options Market

Another factor to look into is the options market. Although the options market doesn't directly tell us how a stock is performing, the option traders may give us a hint as to where the stock is heading in a short period of time. If the ratio of puts to calls is high, the stock price will most likely drop. If the ratio of calls to puts is high, the stock price will most likely jump.

### Put to Call Ratio ###
- Divide sum of puts by sum of calls
- More calls than puts means the asset price will increase (as investors want to buy the asset at a predetermined price) --> they are expecting the price to increase
- More puts than calls means the asset price will decrease (as investors want to sell the asset at a predetermined price) --> they are expecting the price to fall

### Checking if the options market is large enough for the chosen stocks:

In [ ]:
# Global variables to check for availability of options, put-to-call ratios and call-to-put ratios
options_large_enough = False
pcr_large_enough = False
cpr_large_enough = False

In [ ]:
options_evaluation_raw = {}
options_evaluation_raw['puts_sum'] = {}
options_evaluation_raw['calls_sum'] = {}
options_evaluation_raw['pcr'] = {}
options_evaluation_raw['cpr'] = {}

def extract_options_market(ticker):

    try:
        asset = yf.Ticker(ticker)
        options = asset.option_chain(asset.options[0])

        puts = pd.DataFrame().append(options.puts)
        puts_sum = puts['volume'].sum()
        options_evaluation_raw["puts_sum"][ticker] = puts_sum

        calls = pd.DataFrame().append(options.calls)
        calls_sum = calls['volume'].sum()
        options_evaluation_raw["calls_sum"][ticker] = calls_sum

        puts_to_calls =  puts_sum / calls_sum
        options_evaluation_raw["pcr"][ticker] = puts_to_calls

        calls_to_puts =  calls_sum / puts_sum
        options_evaluation_raw["cpr"][ticker] = calls_to_puts
    
    except IndexError as error:
        print(f'Dropped {ticker}, no options data available')
    
# Checking options market of each ticker in list of tickers given from threading
for ticker in tickers:
    thread = Thread(target=extract_options_market, args=[ticker])
    thread.start()
    threads.append(thread)

# Using threading
for thread in threads:
    thread.join()        


Dropped MON, no options data available


In [ ]:
if len(options_evaluation_raw.keys()) <= 12:
    options_large_enough = True
    options_evaluation = pd.DataFrame(options_evaluation_raw)
    print("The options market was large enough for at least 12 stocks. Here is the summary:")
    display(options_evaluation.head())
else:
    print("There was insufficient options data for the tickers provided.")

The options market was large enough for at least 12 stocks. Here is the summary:


,puts_sum,calls_sum,pcr,cpr
BK,12.0,21.0,0.571429,1.750000
USB,257.0,64.0,4.015625,0.249027
PM,60.0,275.0,0.218182,4.583333
KMI,665.0,1106.0,0.601266,1.663158
MO,567.0,228.0,2.486842,0.402116


Let's filter for any stocks with a put-to-call ratio which is below one, and filter for any stocks with a call-to-put ratio which is below one separately. We want our portfolio to move large in one direction, so we must account for stocks which may offset any gains and losses from other stocks. Thus, we will create a portfolio strictly consisting of heavy put-to-call ratio stocks and another portfolio strictly consisting of heavy call-to-put ratio stocks for further comparison. If only one portfolio can be formed, then the comparison step will be skipped.

In [ ]:
if options_large_enough:
    # Filtering for stocks with a puts-to-calls ratio greater than 1 and sorting
    pcr_filter = pd.DataFrame()
    pcr_filter = options_evaluation[options_evaluation["pcr"] > 1].dropna()
    pcr_filter = pcr_filter.sort_values(by=['pcr'], ascending=False)
    print("Displaying the puts-to-calls ratios for the stocks:")
    display(pcr_filter.head())
else:
    print("The options market had insufficient data, skipping...")

Displaying the puts-to-calls ratios for the stocks:


,puts_sum,calls_sum,pcr,cpr
SLB,2559.0,317.0,8.072555,0.123877
TXN,1601.0,222.0,7.211712,0.138663
AAPL,57121.0,11785.0,4.846924,0.206316
PYPL,2935.0,655.0,4.480916,0.223169
AXP,778.0,180.0,4.322222,0.231362


Forming the portfolio for the heavy put-to-call ratio stocks,

In [ ]:
# Checking if there are enough heavy puts-to-calls ratio stocks
if options_large_enough and len(pcr_filter) >= 12:
    print("There are at least 12 stocks with a heavy puts-to-calls ratio.")
    pcr_large_enough = True
    pcr_heavy_tickers = []
    pcr_heavy_tickers = list(pcr_filter.index.values)
    # Removing everything after the first 12 tickers
    pcr_heavy_tickers = pcr_heavy_tickers[0:12]
    print(pcr_heavy_tickers)
else: 
    print("There are not enough stocks with a puts-to-calls ratio greater than 1.")


There are at least 12 stocks with a heavy puts-to-calls ratio.
['SLB', 'TXN', 'AAPL', 'PYPL', 'AXP', 'USB', 'UNP', 'NKE', 'CAT', 'MO', 'SPG', 'BMY']


In [ ]:
if options_large_enough:
    # Filtering for stocks with a calls-to-puts ratio greater than 1
    cpr_filter = pd.DataFrame()
    cpr_filter = options_evaluation[options_evaluation["cpr"] > 1].dropna()
    cpr_filter = cpr_filter.sort_values(by=['cpr'], ascending=False)
    print("Displaying the calls-to-puts ratios for the stocks:")
    display(cpr_filter.head())
else:
    print("The options market had insufficient data, skipping...")

Displaying the calls-to-puts ratios for the stocks:


,puts_sum,calls_sum,pcr,cpr
AIG,193.0,4247.0,0.045444,22.005181
NEE,443.0,6556.0,0.067572,14.799097
MS,3945.0,24982.0,0.157914,6.332573
PM,60.0,275.0,0.218182,4.583333
SBUX,182.0,778.0,0.233933,4.274725


In [ ]:
# Checking if there are enough heavy calls-to-puts ratio stocks
if options_large_enough and len(cpr_filter) >= 12:
    print("There are at least 12 stocks with a heavy calls-to-puts ratio.")
    cpr_large_enough = True
    cpr_heavy_tickers = []
    cpr_heavy_tickers = list(cpr_filter.index.values)
    # Removing everything after the first 12 tickers
    cpr_heavy_tickers = cpr_heavy_tickers[0:12]
    print(cpr_heavy_tickers)
else: 
    print("There are not enough stocks with a calls-to-puts ratio greater than 1.")


There are at least 12 stocks with a heavy calls-to-puts ratio.
['AIG', 'NEE', 'MS', 'PM', 'SBUX', 'PFE', 'SO', 'COF', 'OXY', 'C', 'BAC', 'UPS']


#### Below is the function that calculates the number of shares we will invest in each stock given a list of tickers ####
So far, we have obtained the tickers from the methods we used above and gotten a couple of contestents for our final portfolio. Now, we will be throwing them into the function below to yield the amount of stocks we will invest into each ticker for each portfolio.

In [ ]:
# This function calculates the number of stocks to invest in given a list of 12 stocks
def num_stocks (tickers_list):
    
    # Calculating the $ amount to invest into each stock 
    investment_list = []
    for i in range(len(list_of_weighting)):
        investment_list.append((list_of_weighting[i] * investment))
        
    # Getting the number of stocks invested in each ticker
    stocks_in_each = []
    for i in range(len(tickers_list)):
        stocks_in_each.append (investment_list[i] / tickers_hist.get(tickers_list[i])['Close'][0])
    
    frame = {"Ticker": tickers_list,
             "Stocks": stocks_in_each,
             "Investment": investment_list,
             "Weighting": list_of_weighting}
    
    stocks_dataframe = pd.DataFrame(frame)

    return (stocks_dataframe)

In [ ]:
beta_portfolio = num_stocks(beta_tickers)
beta_portfolio 

,Ticker,Stocks,Investment,Weighting
0,PYPL,641.222932,125000.000000,0.250000
1,AMZN,733.548724,125000.000000,0.250000
2,QCOM,335.642542,62500.000000,0.125000
3,GM,340.580905,20833.333333,0.041667
4,GOOG,143.604385,20833.333333,0.041667
5,NKE,126.515659,20833.333333,0.041667
6,BA,100.227717,20833.333333,0.041667
7,BLK,22.850333,20833.333333,0.041667
8,AAPL,114.462579,20833.333333,0.041667
9,MSFT,62.235499,20833.333333,0.041667


In [ ]:
# Helper function before generating the entire portfolio
# generate_portfolio takes in a ticker and tracks how the ticker does over a time period
def generate_portfolio(ticker, num_stocks):
    
    # Finding the number of trading days for the stock
    ticker_hist = yf.Ticker(ticker).history(start=start_date, end=end_date, interval='1d').dropna() 
    days_trading = len(ticker_hist.index)
        
    
    # Calculating how the stock performs over this time period
    values = []
    for i in range(days_trading):
        value = tickers_hist.get(ticker)['Close'][i] * num_stocks
        values.append(value)
        
        
    return values
    
# generate_combined_portfolio takes in a dataframe (index, "Ticker", "Stock", "Investment") and generates the portfolio over the time period
def generate_combined_portfolio(dataframe):
    final_df = pd.DataFrame()
    values_list = []
    ticker_list = []

    for i in range(len(dataframe.index)):
        ticker = dataframe['Ticker'][i]
        ticker_list.append(ticker)
        values_list.append(generate_portfolio(ticker, dataframe['Stocks'][i]))

    for i in range(len(dataframe.index)):
        frame = {ticker_list[i]: values_list[i]}
        final_df[ticker_list[i]] = pd.DataFrame(frame)

    final_df = final_df.set_index(tickers_hist.get(dataframe['Ticker'][0]).index)
    final_df['Total Value'] = final_df.sum(axis=1)
    
    return final_df

### Calculating betas for the portfolios with heavy calls-to-puts ratios and heavy puts-to-calls ratios

In [ ]:
if cpr_large_enough:
    # Getting the call-to-put ratio portfolios' betas
    cpr_df = pd.DataFrame(cpr_heavy_tickers, columns = ['Ticker'])
    cpr_df['Beta'] = cpr_df['Ticker'].replace(beta_dict)

    cpr_df.sort_values(by = ['Beta'], ascending=False, ignore_index = True, inplace = True)
    display(cpr_df)
else:
    print("The call-to-put ratio was not available for enough stocks.")

,Ticker,Beta
0,COF,1.247706
1,SBUX,1.107557
2,MS,1.052367
3,AIG,1.008040
4,BAC,0.991630
5,C,0.949343
6,UPS,0.850226
7,NEE,0.744378
8,OXY,0.703407
9,PFE,0.528784


In [ ]:
if pcr_large_enough:
    # Getting the put-to-call ratio portfolios' betas
    pcr_df = pd.DataFrame(pcr_heavy_tickers, columns = ['Ticker'])
    pcr_df['Beta'] = pcr_df['Ticker'].replace(beta_dict)

    pcr_df.sort_values(by = ['Beta'], ascending=False, ignore_index = True, inplace = True)

    display(pcr_df)
else:
    print("The put-to-call ratio was not available for enough stocks.")

The put-to-call ratio was not available for enough stocks.


In [ ]:
if (cpr_large_enough and pcr_large_enough):
    # Generating portfolios for PCR and CPR stocks
    pcr_list = pcr_df['Ticker'].tolist()
    pcr_portfolio = generate_combined_portfolio(num_stocks(pcr_list))
    cpr_list = cpr_df['Ticker'].tolist()
    cpr_portfolio = generate_combined_portfolio(num_stocks(cpr_list))

    # Graphing the portfolios
    plt.figure(figsize=(20, 5))
    plt.plot(pcr_portfolio.index, pcr_portfolio['Total Value'], label='PCR Portfolio')
    plt.plot(cpr_portfolio.index, cpr_portfolio['Total Value'], label='CPR Portfolio')

    plt.title('PCR vs CPR Portfolio')
    plt.xlabel('Dates')
    plt.ylabel('Value')

    plt.legend()
    plt.show()

elif(cpr_large_enough):
    # Generating protfolio for CPR stocks
    cpr_list = cpr_df['Ticker'].tolist()
    cpr_portfolio = generate_combined_portfolio(num_stocks(cpr_list))

    # Graphing the portfolio
    plt.figure(figsize=(20, 5))
    plt.plot(cpr_portfolio.index, cpr_portfolio['Total Value'], label='CPR Portfolio')

    plt.title('CPR Portfolio')
    plt.xlabel('Dates')
    plt.ylabel('Value')

    plt.legend()
    plt.show()

elif(pcr_large_enough):
    # Gererating portfolios for PCR stocks
    pcr_list = pcr_df['Ticker'].tolist()
    pcr_portfolio = generate_combined_portfolio(num_stocks(pcr_list))

    # Graphing the portfolios
    plt.figure(figsize=(20, 5))
    plt.plot(pcr_portfolio.index, pcr_portfolio['Total Value'], label='PCR Portfolio')

    plt.title('PCR Portfolio')
    plt.xlabel('Dates')
    plt.ylabel('Value')

    plt.legend()
    plt.show()
else:
    print ("None of the options markets were large enough.")


None of the options markets were big enough.


### Comparing PCR and CPR Betas ###
The above should graph the PCR and CPR portfolios. In the case that only one of them is available, it will only print the available one. If none are available, then this will be skipped.

In [ ]:
if (cpr_large_enough and pcr_large_enough):
    cpr_beta_mean = cpr_df['Beta'].mean()
    pcr_beta_mean = pcr_df['Beta'].mean()
elif (cpr_large_enough):
    cpr_beta_mean = cpr_df['Beta'].mean()
elif (pcr_large_enough):
    pcr_beta_mean = pcr_df['Beta'].mean()
else:
    print("None of the options markets were large enough")


None of the options markets were large enough


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.